In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
path_to_git = '/Users/harshparikh/Documents/GitHub/iic_causal_inference/'

In [4]:
iic_burden = pd.read_pickle(path_to_git + 'aim1/data_to_fit_CNNIIC_iic_burden_smooth.pickle')

In [18]:
iic_burden.keys()

dict_keys(['W', 'window_start_ids', 'D', 'Ddose', 'Dname', 'Pobs', 'Pname', 'C', 'Cname', 'Y', 'Yname', 'cluster', 'sids', 'pseudoMRNs'])

In [30]:
iic_burden['D'][0].shape

(197, 7)

In [27]:
def plot_unit(E,Ename,D,Dnames):
    n_drugs = D.shape[1]
    drug_names = Dnames
    fig,axs = plt.subplots(nrows=1+D.shape[1],ncols=1,
                           sharex=True,
                           figsize=(15,5+0.5*n_drugs),
                           gridspec_kw = {'height_ratios':[10]+[1 for i in range(D.shape[1])]})
    ax[0].plot( np.arange(0,T), E, c='black',label='Observed')
    

(197,)
(167,)
(158,)
(173,)
(69,)
(192,)
(178,)
(158,)
(157,)
(183,)
(140,)
(193,)
(154,)
(200,)
(162,)
(154,)
(167,)
(184,)
(153,)
(170,)
(183,)
(133,)
(199,)
(142,)
(182,)
(202,)
(157,)
(162,)
(178,)
(194,)
(184,)
(188,)
(208,)
(200,)
(145,)
(190,)
(148,)
(160,)
(192,)
(161,)
(167,)
(156,)
(157,)
(186,)
(199,)
(191,)
(218,)
(181,)
(40,)
(169,)
(154,)
(177,)
(123,)
(150,)
(169,)
(145,)
(102,)
(130,)
(37,)
(154,)
(150,)
(167,)
(165,)
(150,)
(149,)
(53,)
(149,)
(125,)
(153,)
(170,)
(126,)
(166,)
(136,)
(147,)
(80,)
(169,)
(171,)
(160,)
(160,)
(160,)
(149,)
(190,)
(152,)
(175,)
(145,)
(151,)
(217,)
(156,)
(150,)
(209,)
(171,)
(141,)
(102,)
(140,)
(35,)
(154,)
(202,)
(130,)
(211,)
(66,)
(159,)
(164,)
(149,)
(154,)
(165,)
(195,)
(142,)
(187,)
(162,)
(120,)
(156,)
(178,)
(107,)
(193,)
(171,)
(171,)
(190,)
(148,)
(166,)
(148,)
(151,)
(161,)
(113,)
(203,)
(144,)
(221,)
(159,)
(155,)
(199,)
(126,)
(170,)
(134,)
(68,)
(205,)
(131,)
(176,)
(153,)
(214,)
(145,)
(173,)
(172,)
(168,)
(193,)
(168,)
